# Import necessary dependencies

In [1]:
import pandas as pd
import numpy as np
import text_normalizer as tn
import warnings
import nltk

warnings.filterwarnings("ignore")

# Load and normalize data

In [3]:
dataset = pd.read_csv(r'movie_reviews.csv')

# take a peek at the data
print(dataset.head())
reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

# build train and test datasets
train_reviews = reviews[:35000]
train_sentiments = sentiments[:35000]
test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]

# normalize datasets
stop_words = nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('but')
stop_words.remove('not')

norm_train_reviews = tn.normalize_corpus(train_reviews, stopwords=stop_words)
norm_test_reviews = tn.normalize_corpus(test_reviews, stopwords=stop_words)

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


# Extract features from positive and negative reviews

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# consolidate all normalized reviews
norm_reviews = norm_train_reviews+norm_test_reviews
# get tf-idf features for only positive reviews
positive_reviews = [review for review, sentiment in zip(norm_reviews, sentiments) if sentiment == 'positive']
ptvf = TfidfVectorizer(use_idf=True, min_df=0.02, max_df=0.75, ngram_range=(1, 2), sublinear_tf=True)
ptvf_features = ptvf.fit_transform(positive_reviews)
# get tf-idf features for only negative reviews
negative_reviews = [review for review, sentiment in zip(norm_reviews, sentiments) if sentiment == 'negative']
ntvf = TfidfVectorizer(use_idf=True, min_df=0.02, max_df=0.75, ngram_range=(1, 2), sublinear_tf=True)
ntvf_features = ntvf.fit_transform(negative_reviews)
# view feature set dimensions
print(ptvf_features.shape, ntvf_features.shape)

(25000, 933) (25000, 925)


# Topic Modeling on Reviews

In [5]:
import pyLDAvis
import pyLDAvis.sklearn
from sklearn.decomposition import NMF
import topic_model_utils as tmu

pyLDAvis.enable_notebook()
total_topics = 10

## Display and visualize topics for positive reviews

In [6]:
# build topic model on positive sentiment review features
pos_nmf = NMF(n_components=total_topics, solver='cd', max_iter=500,
               random_state=42, alpha=.1, l1_ratio=.85)
pos_nmf.fit(ptvf_features)      
# extract features and component weights
pos_feature_names = np.array(ptvf.get_feature_names())
pos_weights = pos_nmf.components_
# extract and display topics and their components
pos_feature_names = np.array(ptvf.get_feature_names())
feature_idxs = np.argsort(-pos_weights)[:, :15]
topics = [pos_feature_names[idx] for idx in feature_idxs]
for idx, topic in enumerate(topics):
    print('Topic #'+str(idx+1)+':')
    print(', '.join(topic))
    print()

Topic #1:
but, one, make, no, take, way, even, get, seem, like, much, scene, may, character, go

Topic #2:
movie, watch, see, like, think, really, good, but, see movie, great, movie not, would, get, enjoy, say

Topic #3:
show, episode, series, tv, season, watch, dvd, television, first, good, see, would, air, great, remember

Topic #4:
family, old, young, year, life, child, father, mother, son, year old, man, friend, kid, boy, girl

Topic #5:
performance, role, actor, play, great, cast, good, well, excellent, character, story, star, also, give, acting

Topic #6:
film, see, see film, film not, watch, good film, watch film, dvd, great film, film but, film see, release, year, film make, great

Topic #7:
love, love movie, story, love story, fall love, fall, beautiful, song, wonderful, music, heart, romantic, romance, favorite, character

Topic #8:
funny, laugh, hilarious, joke, humor, moment, fun, guy, get, but, line, show, lot, time, scene

Topic #9:
ever, ever see, movie ever, one good, o

In [7]:
pyLDAvis.sklearn.prepare(pos_nmf, ptvf_features, ptvf, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.146073  0.111110       1        1  31.829570
1      0.001605  0.058167       2        1  14.885487
4     -0.211286  0.289776       3        1  12.508661
3      0.139620 -0.143214       4        1  10.200667
5      0.388178 -0.010067       5        1   9.089744
2      0.212107  0.143506       6        1   6.375484
6      0.144678  0.440802       7        1   5.237892
8      0.180189 -0.430785       8        1   3.912696
7     -0.418456 -0.102275       9        1   3.359701
9     -0.290561 -0.357022      10        1   2.600098, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
144   Default  2165.000000       comedy  2165.000000  30.0000  30.0000
464   Default  2209.000000         love  2209.000000  29.0000  29.0000
334   Default  1760.000000        funny  1760.000000  28.0000  28.0000
299   Default  2521.000000         film  2521.000000  27.0000  27.0000
254   Default  1556.000000         ever  1556.000000  26.0000  26.0000
436   Default  1107.000000        laugh  1107.000000  25.0000  25.0000
256   Default   848.000000     ever see   848.000000  24.0000  24.0000
705   Default  1819.000000          see  1819.000000  23.0000  23.0000
730   Default  1188.000000         show  1188.000000  22.0000  22.0000
245   Default   817.000000      episode   817.000000  21.0000  21.0000
512   Default  1669.000000        movie  1669.000000  20.0000  20.0000
514   Default   636.000000   movie ever   636.000000  19.0000  19.0000
716   Default   738.000000       series   738.000000  18.0000  18.0000
573   Default  1379.000000          one  1379.000000  17.0000  17.0000
574   Default   662.000000     one good   662.000000  16.0000  16.0000
350   Default  1540.000000         good  1540.000000  15.0000  15.0000
380   Default   606.000000    hilarious   606.000000  14.0000  14.0000
688   Default   499.000000     romantic   499.000000  13.0000  13.0000
780   Default   990.000000        story   990.000000  12.0000  12.0000
465   Default   456.000000   love movie   456.000000  11.0000  11.0000
706   Default   506.000000     see film   506.000000  10.0000  10.0000
889   Default  1147.000000        watch  1147.000000   9.0000   9.0000
280   Default   551.000000       family   551.000000   8.0000   8.0000
598   Default   610.000000  performance   610.000000   7.0000   7.0000
857   Default   532.000000           tv   532.000000   6.0000   6.0000
466   Default   373.000000   love story   373.000000   5.0000   5.0000
277   Default   431.000000         fall   431.000000   4.0000   4.0000
333   Default   515.000000          fun   515.000000   3.0000   3.0000
278   Default   353.000000    fall love   353.000000   2.0000   2.0000
686   Default   549.000000         role   549.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
95    Topic10    53.543593      british   143.573914   2.6633  -4.4368
339   Topic10    41.020628       genius   118.507537   2.5887  -4.7032
333   Topic10   146.161057          fun   515.044723   2.3901  -3.4325
380   Topic10   169.809320    hilarious   606.099575   2.3773  -3.2826
395   Topic10   104.944827        humor   388.224116   2.3415  -3.7638
340   Topic10    40.690285        genre   157.233005   2.2979  -4.7113
732   Topic10    30.218190        silly   124.038411   2.2375  -5.0088
723   Topic10    32.818701          sex   135.318019   2.2330  -4.9263
813   Topic10    27.448512         team   118.212398   2.1895  -5.1049
214   Topic10    63.697005        drama   278.189615   2.1755  -4.2631
687   Topic10    56.534516      romance   262.765213   2.1132  -4.3824
124   Topic10    26.469988     charming   131.194532   2.0490  -5.1412
84    Topic10    30.011825        black   150.400192   2.0379  -5.0157
717   Topic10    26.369110      serious   1

## Display and visualize topics for negative reviews

In [8]:
# build topic model on negative sentiment review features
neg_nmf = NMF(n_components=total_topics, solver='cd', max_iter=500,
              random_state=42, alpha=.1, l1_ratio=.85)
neg_nmf.fit(ntvf_features)      
# extract features and component weights
neg_feature_names = ntvf.get_feature_names()
neg_weights = neg_nmf.components_
# extract and display topics and their components
neg_feature_names = np.array(ntvf.get_feature_names())
feature_idxs = np.argsort(-neg_weights)[:, :15]
topics = [neg_feature_names[idx] for idx in feature_idxs]
for idx, topic in enumerate(topics):
    print('Topic #'+str(idx+1)+':')
    print(', '.join(topic))
    print()

Topic #1:
but, one, character, get, go, like, no, scene, seem, take, show, much, time, would, play

Topic #2:
movie, watch, good, bad, think, like, but, see, would, make, even, movie not, could, really, watch movie

Topic #3:
film, film not, good, bad, make, bad film, acting, film but, but, actor, watch film, see film, script, watch, see

Topic #4:
horror, budget, low, low budget, horror movie, horror film, gore, flick, zombie, blood, scary, killer, monster, kill, genre

Topic #5:
effect, special, special effect, fi, sci, sci fi, acting, bad, look, look like, cheesy, terrible, cheap, creature, space

Topic #6:
funny, comedy, joke, laugh, not funny, show, humor, stupid, try, hilarious, but, fun, suppose, episode, moment

Topic #7:
ever, ever see, bad, bad movie, movie ever, see, one bad, ever make, one, movie, film ever, bad film, make, horrible, movie bad

Topic #8:
waste, waste time, time, not waste, money, complete, hour, spend, life, talent, please, crap, total, plot, minute

Topic 

In [9]:
pyLDAvis.sklearn.prepare(neg_nmf, ntvf_features, ntvf, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.075880  0.012111       1        1  40.232987
1     -0.015644 -0.107888       2        1  17.717063
2     -0.190566 -0.059404       3        1  11.110715
6     -0.244029  0.294654       4        1   5.358274
3      0.198677 -0.396170       5        1   5.129150
5      0.436847 -0.099918       6        1   4.872058
9      0.079129  0.485707       7        1   4.674379
7     -0.246012 -0.423321       8        1   4.109051
4     -0.458788  0.039139       9        1   3.503540
8      0.364508  0.255091      10        1   3.292783, topic_info=     Category         Freq            Term        Total  loglift  logprob
term                                                                     
877   Default  1763.000000           waste  1763.000000  30.0000  30.0000
84    Default  1635.000000            book  1635.000000  29.0000  29.0000
330   Default  1739.000000           funny  1739.000000  28.0000  28.0000
918   Default  1653.000000            year  1653.000000  27.0000  27.0000
580   Default  1608.000000             old  1608.000000  26.0000  26.0000
298   Default  2285.000000            film  2285.000000  25.0000  25.0000
878   Default  1351.000000      waste time  1351.000000  24.0000  24.0000
233   Default  1214.000000          effect  1214.000000  23.0000  23.0000
827   Default  1398.000000            time  1398.000000  22.0000  22.0000
765   Default  1092.000000         special  1092.000000  21.0000  21.0000
257   Default  1267.000000            ever  1267.000000  20.0000  20.0000
920   Default  1169.000000        year old  1169.000000  19.0000  19.0000
766   Default  1025.000000  special effect  1025.000000  18.0000  18.0000
387   Default  1143.000000          horror  1143.000000  17.0000  17.0000
144   Default  1136.000000          comedy  1136.000000  16.0000  16.0000
660   Default   987.000000            read   987.000000  15.0000  15.0000
54    Default  1869.000000             bad  1869.000000  14.0000  14.0000
259   Default   895.000000        ever see   895.000000  13.0000  13.0000
58    Default   935.000000       bad movie   935.000000  12.0000  12.0000
561   Default   797.000000       not waste   797.000000  11.0000  11.0000
418   Default   825.000000            joke   825.000000  10.0000  10.0000
507   Default   827.000000      movie ever   827.000000   9.0000   9.0000
434   Default   938.000000           laugh   938.000000   8.0000   8.0000
97    Default   827.000000          budget   827.000000   7.0000   7.0000
543   Default   713.000000       not funny   713.000000   6.0000   6.0000
461   Default   769.000000             low   769.000000   5.0000   5.0000
503   Default  1759.000000           movie  1759.000000   4.0000   4.0000
498   Default   689.000000           money   689.000000   3.0000   3.0000
462   Default   639.000000      low budget   639.000000   2.0000   2.0000
710   Default  1416.000000             see  1416.000000   1.0000   1.0000
...       ...          ...             ...          ...      ...      ...
627   Topic10    45.969686         portray   114.952814   2.4969  -4.8468
288   Topic10    29.474462        favorite    81.144326   2.4007  -5.2912
194   Topic10    22.623377          detail    67.627426   2.3184  -5.5558
145   Topic10    53.819388           comic   163.316595   2.3034  -4.6891
780   Topic10   248.125720           story   781.126245   2.2666  -3.1608
494   Topic10    53.280903            miss   169.842372   2.2541  -4.6992
841   Topic10    46.408148            true   149.084162   2.2464  -4.8373
368   Topic10    26.679472           heart    86.187017   2.2408  -5.3908
515   Topic10    39.543039              mr   128.006842   2.2387  -4.9974
200   Topic10    46.051941       different   154.003706   2.2062  -4.8450
706   Topic10    25.751229      screenplay    86.608445   2.2005  -5.4263
205   Topic10    41.577245    disappointed   140.0878